In [4]:
import pandas as pd
import h5py
import os

In [184]:
# 4k Skims
working_dir = r'\\MODELSRV8\d$\4K_V405'
h5file = '2014-travelmodel.h5'
myh5 = h5py.File(os.path.join(working_dir,h5file))

df_final = pd.DataFrame()

for year in ['2014','2025','2040']:
    h5_4k = h5py.File(r'\\MODELSRV8\d$\4K_V405\\'+year+'-travelmodel.h5')
    h5_sc = h5py.File(r'R:\SoundCast\UrbanSimAccessibility\Skims\\'+year+'\\urbansim_skims.h5')
    for matrix_name in myh5['results'].keys():
#         print matrix_name

        # Load 4k skims
        source = '4k'
        _df_4k = pd.DataFrame(h5_4k['results'][matrix_name][:])
        # Adjust index to start from zone 1 to 3700
        _df_4k.index = _df_4k.index+1
        _df_4k.columns = _df_4k.columns+1
        
        max_4k = _df_4k.max().max()
        
        # Load Soundcast skims
        _df_sc = pd.DataFrame(h5_sc['results'][matrix_name][:])
        # Adjust index to start from zone 1 to 3700
        _df_sc.index = _df_sc.index+1
        _df_sc.columns = _df_sc.columns+1
        
        max_sc = _df_sc.max().max()
        
#         # Get max of the matrix and use min between 4k and sc to scale down
#         # This eliminates odd sum values when max values are very different
#         # e.g., 4k uses max walk time sa 10e20 while soundcast uses 360 
        scale_max = min(max_sc,max_4k)
        
#         _df_4k[_df_4k > scale_max]  = scale_max
#         _df_sc[_df_sc > scale_max]  = scale_max

        # Alternatively, find where one matrix has these max values and set all of them to 0
        # use mask for each matrix
        _df_sc[_df_4k > scale_max] = 0
        _df_sc[_df_sc > scale_max] = 0
        _df_4k[_df_sc > scale_max] = 0
        _df_4k[_df_4k > scale_max] = 0
        
        # Sum skim/trip values from origin
        df_4k = pd.DataFrame(_df_4k.unstack()).reset_index()
        df_4k = df_4k.rename(columns={'level_0': 'otaz', 'level_1': 'dtaz', 0: 'trips'})

        # Get origin totals from otaz (too many records for full O-D comparison)
        df_4k = df_4k.groupby('otaz').sum()[['trips']]
        df_4k = df_4k.reset_index()

        # Process Soundcast skims
        source = 'soundcast'
        
        df_sc = pd.DataFrame(_df_sc.unstack()).reset_index()
        df_sc = df_sc.rename(columns={'level_0': 'otaz', 'level_1': 'dtaz', 0: 'trips'})

        # Get total trips from otaz (too many records for full O-D comparison)
        df_sc = df_sc.groupby('otaz').sum()[['trips']]
        df_sc = df_sc.reset_index()

        # Merge results
        df = pd.merge(df_4k,df_sc,on='otaz', suffixes=['_4k','_sc'])
        df['matrix_name'] = matrix_name
        df['year'] = year
    
        df_final = df_final.append(df)

In [178]:
# df_final

In [185]:
df_final.to_csv(r'R:\SoundCast\UrbanSimAccessibility\skim_comparison.csv', index=False)

## 4k Skims

In [173]:
test = pd.DataFrame(myh5['results']['atrtwa'][:])

In [183]:
test > 300

,0,1,2,3,4,5,6,7,8,9,...,3690,3691,3692,3693,3694,3695,3696,3697,3698,3699
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,True,True,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,True,True,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True


In [175]:
test.max()

0       1.000000e+20
1       1.000000e+20
2       1.000000e+20
3       1.000000e+20
4       1.000000e+20
5       1.000000e+20
6       1.000000e+20
7       1.000000e+20
8       1.000000e+20
9       1.000000e+20
10      1.000000e+20
11      1.000000e+20
12      1.000000e+20
13      1.000000e+20
14      1.000000e+20
15      1.000000e+20
16      1.000000e+20
17      1.000000e+20
18      1.000000e+20
19      1.000000e+20
20      1.000000e+20
21      1.000000e+20
22      1.000000e+20
23      1.000000e+20
24      1.000000e+20
25      1.000000e+20
26      1.000000e+20
27      1.000000e+20
28      1.000000e+20
29      1.000000e+20
            ...     
3670    1.000000e+20
3671    1.000000e+20
3672    1.000000e+20
3673    1.000000e+20
3674    1.000000e+20
3675    1.000000e+20
3676    1.000000e+20
3677    1.000000e+20
3678    1.000000e+20
3679    1.000000e+20
3680    1.000000e+20
3681    1.000000e+20
3682    1.000000e+20
3683    1.000000e+20
3684    1.000000e+20
3685    1.000000e+20
3686    1.000

In [113]:
working_dir = r'\\MODELSRV8\d$\4K_V405'
h5file = '2014-travelmodel.h5'
myh5 = h5py.File(os.path.join(working_dir,h5file))

In [114]:
matrix_name = 'aau1cs'
source = '4k'
df = pd.DataFrame(myh5['results'][matrix_name][:])
# Adjust index to start from zone 1 to 3700
df.index = df.index+1
df.columns = df.columns+1
df_4k = pd.DataFrame(df.unstack()).reset_index()
df_4k = df_4k.rename(columns={'level_0': 'otaz', 'level_1': 'dtaz', 0: 'trips'})

# Get total trips from otaz (too many records for full O-D comparison)
df_4k = df_4k.groupby('otaz').sum()[['trips']]
df_4k = df_4k.reset_index()

## Soundcast Skims

In [115]:
working_dir = r'\\file2\e2projects_two\SoundCast\UrbanSimAccessibility\Skims\2014'
h5file = 'urbansim_skims.h5'
myh5 = h5py.File(os.path.join(working_dir,h5file))

In [116]:
matrix_name = 'aau1cs'
source = 'soundcast'
df = pd.DataFrame(myh5['results'][matrix_name][:])
# Adjust index to start from zone 1 to 3700
df.index = df.index+1
df.columns = df.columns+1
df_sc = pd.DataFrame(df.unstack()).reset_index()
df_sc = df_sc.rename(columns={'level_0': 'otaz', 'level_1': 'dtaz', 0: 'trips'})

# Get total trips from otaz (too many records for full O-D comparison)
df_sc = df_sc.groupby('otaz').sum()[['trips']]
df_sc = df_sc.reset_index()

In [117]:
# Join

In [118]:
df = pd.merge(df_4k,df_sc,on='otaz', suffixes=['_4k','_sc'])
df['matrix_name'] = matrix_name

In [119]:
df.to_csv(r'c:/users/brice/skim_compare.csv', index=False)

In [121]:
df

,otaz,trips_4k,trips_sc,matrix_name
0,1,210246.875000,223922.578125,aau1cs
1,2,206661.781250,220804.078125,aau1cs
2,3,204097.750000,218880.125000,aau1cs
3,4,202862.265625,217652.328125,aau1cs
4,5,201796.234375,216923.062500,aau1cs
5,6,201030.484375,216746.125000,aau1cs
6,7,200872.390625,216489.500000,aau1cs
7,8,197454.000000,213889.031250,aau1cs
8,9,198494.968750,214974.062500,aau1cs
9,10,197906.140625,214410.062500,aau1cs
